<div>
<img src="figures/svtLogo.png"/>
</div>
<h1><center>Mathematical Optimization for Engineers</center></h1>
<h2><center>Lab 14 - Uncertainty</center></h2>

We want to optimize the total annualized cost of a heating and electric power system. Three different technologies are present: 
- a gas boiler
- a combined heat and power plant
- a photovoltaic module

We first the the nominal case without uncertanties. 
Next, we will consider a two-stage approach to consider uncertainties in the electricity demand and the power producable via PV. 
Uncertain variables are the solar power and the power demand. 

In [21]:
# import cell
from scipy.optimize import minimize, NonlinearConstraint, Bounds

In [22]:
class Boiler():
    """Boiler 
    Gas in, heat out
    """
    
    def __init__(self):
        self.M = 0.75  
        
    def invest_cost(self, Qdot_nom):
        inv = 100 * Qdot_nom ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas = 60
        cost_gas_oper = Qdot_nom * cost_gas * op_load
        
        return cost_gas_oper
    
    def heat(self, Qdot_nom, op_load):
        eta_th = 0.9 - (1 - op_load) * 0.05
        return Qdot_nom * op_load * eta_th
    

In [23]:
class CHP():
    """Combined-heat-and-power (CHP) engine 
    Gas in, heat and power out
    """

    def __init__(self):
        self.c_ref = 150
        self.M = 0.80  # [-], cost exponent
        self.cost_gas = 60
    
    def invest_cost(self, Qdot_nom):
        inv = self.c_ref * (Qdot_nom) ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas_oper = Qdot_nom * op_load * self.cost_gas
        return cost_gas_oper
    
    def elec_out(self, Qdot_nom, op_load):
        eta_el = 0.3 - (1 - op_load) * 0.1
        out_pow = eta_el * Qdot_nom * op_load
        return out_pow
    
    def heat(self, Qdot_nom, op_load): 
        eta_th = 0.6 - (1-op_load) * 0.05  
        return Qdot_nom * eta_th * op_load


In [24]:
class PV:
    """Photovoltaic modules (PV) 
    solar 
    """ 
    
    def __init__(self): 
        self.M = 0.9  # [-], cost exponent
       
    def invest_cost(self, p_nom):
        inv = 200 * p_nom ** self.M
        return inv
    
    def oper_cost(self, out_nom): 
        return 0
    
    def elec_out(self, p_nom, op_load, solar):
        return p_nom * op_load * solar
    

In [25]:
def objective_function(x, PV, Boiler, CHP, scenarios):
    total_cost = 0
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 
    
    cost_PV = PV.invest_cost(design_PV) 
    cost_Boiler = Boiler.invest_cost(design_boiler)  
    cost_CHP = CHP.invest_cost(design_CHP) 

    total_cost = cost_PV + cost_Boiler + cost_CHP
    operating_cost = 0
    
    iIndexShift = 3
    for idx, iSec in enumerate(scenarios): 
        indexOffset = 3 + idx * iIndexShift
        
        op_cost = Boiler.oper_cost(design_boiler, x[indexOffset + 1]) \
             + CHP.oper_cost(design_CHP, x[indexOffset + 2])
        total_cost = total_cost + iSec["p"] * op_cost
        
   
    return total_cost

In [26]:
def constraint_function(x, PV, Boiler, CHP, scenarios): 
    heat_demand = 200
    c = list  
    c = [0]
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2]  

    # loop over all uncertatintes
    iIndexShift = 3
    for idx, iSec in enumerate(scenarios): 
        indexOffset = 3 + idx * iIndexShift
        elec_demand = iSec["elec"]
        
        # heat demand
        c.append(Boiler.heat(design_boiler, x[indexOffset + 1]) \
             + CHP.heat(design_CHP, x[indexOffset + 2]) - heat_demand)     
        # electricty demand 
        c.append(PV.elec_out(design_PV, x[indexOffset], iSec["solar"])
              + CHP.elec_out(design_CHP, x[indexOffset + 2]) - elec_demand)
    # remove initial list element       
    c.pop(0)
    return c

In [27]:
def print_solution(x):
    print('PV design: ', x[0])
    print('Boiler design: ', x[1])
    print('CHP design: ', x[2], end='\n\n')

    # print scenarios
    n_scenarios = int((len(x) - 3) / 3)
    for i_scenario in range(1, n_scenarios + 1): 
            print('Scenario ' + str(i_scenario) + ' PV load: ', x[3 * i_scenario])
            print('Scenario ' + str(i_scenario) + ' Boiler load: ', x[3 * i_scenario + 1])
            print('Scenario ' + str(i_scenario) + ' CHP load: ', x[3 * i_scenario + 2], end='\n\n')
   

In [28]:
# nominal case
scenario1 = {"p": 1.0, "solar":1.0, "elec": 100}
scenarios = [scenario1] # some base scenario


In [29]:
# now consider different scenarios
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200, 200, 200, 1, 1, 1]
lbs = [0, 0, 0, 0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1]
bnds = Bounds(lbs, ubs)



In [30]:
res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 25, 'iprint': 2, 'disp': True})


  NIT    FC           OBJFUN            GNORM
    1     8     4.230687E+04     1.697139E+04
    2    15     3.378407E+04     1.515439E+04
    3    22     3.433273E+04     1.216002E+04
    4    29     3.405211E+04     1.113595E+04
    5    36     3.172059E+04     1.148302E+04
    7    43     3.170808E+04     1.333535E+04
   11    49     3.170808E+04     1.445212E+04
Optimization terminated successfully    (Exit mode 0)
            Current function value: 31708.080595121006
            Iterations: 11
            Function evaluations: 49
            Gradient evaluations: 7


In [31]:
print_solution(res.x)

PV design:  100.00000000749218
Boiler design:  222.2222251037179
CHP design:  0.0

Scenario 1 PV load:  1.0
Scenario 1 Boiler load:  1.0
Scenario 1 CHP load:  0.9999999999692892



In [32]:
# nominal 
# uncertanties: power demand and solar power (relative 1.0)
scenario1 = {"p": 0.4, "solar":1.0, "elec": 100}
scenario2 = {"p": 0.3, "solar":1.0, "elec": 120}
scenario3 = {"p": 0.3, "solar":0.5, "elec": 80}

scenarios = [scenario1] # some base scenario
scenarios.append(scenario2)
scenarios.append(scenario3)

print(scenarios)

[{'p': 0.4, 'solar': 1.0, 'elec': 100}, {'p': 0.3, 'solar': 1.0, 'elec': 120}, {'p': 0.3, 'solar': 0.5, 'elec': 80}]


In [33]:
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200,200,200, 1,1,1, 1,1,1, 1,1,1 ]
lbs = [0, 0, 0,   0, 0, 0,  0, 0, 0,  0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1, 1, 1, 1, 1, 1, 1]
bnds = Bounds(lbs, ubs)

res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 15, 'iprint': 2, 'disp': True})

  NIT    FC           OBJFUN            GNORM
    1    14     4.368662E+04     9.896865E+03
    2    27     3.780998E+04     8.491300E+03
    3    40     3.822383E+04     7.547616E+03
    4    53     3.776663E+04     7.318663E+03
    5    66     3.775835E+04     6.635008E+03
    6    79     3.776402E+04     6.598445E+03
    7    92     3.776235E+04     6.599402E+03
    8   113     3.776324E+04     6.599400E+03
   10   126     3.776385E+04     6.599205E+03
   11   139     3.776385E+04     6.599339E+03
   12   152     3.776385E+04     6.599339E+03
   13   165     3.776384E+04     6.599339E+03
   14   188     3.776385E+04     6.599339E+03
   15   200     3.776385E+04     6.599339E+03
Iteration limit reached    (Exit mode 9)
            Current function value: 37763.85488014591
            Iterations: 15
            Function evaluations: 200
            Gradient evaluations: 14


In [34]:
print_solution(res.x)

PV design:  79.9999999973027
Boiler design:  133.3333333364351
CHP design:  133.33333331602304

Scenario 1 PV load:  0.75000000988053
Scenario 1 Boiler load:  1.0
Scenario 1 CHP load:  0.9999999853145068

Scenario 2 PV load:  1.0
Scenario 2 Boiler load:  1.0
Scenario 2 CHP load:  0.9999999928127812

Scenario 3 PV load:  0.999999988561437
Scenario 3 Boiler load:  0.9999999941062396
Scenario 3 CHP load:  1.0



In [20]:
print(objective_function([ 80, 133.333, 133.333, 0.75, 1, 1, 1, 1, 1, 1, 1, 1], myPV, myBoiler, myCHP, scenarios))

[80, 133.333, 133.333]
PV 10323.120194371457
Boiler 3923.767251430592
CHP 7516.945123606054
invest cost 21763.832569408103
37763.7925694081
